In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [204]:
class Simulador_MIP():
    def __init__(self, archivo_domestico, archivo_total):
        self.carga_datos(archivo_domestico, archivo_total)
        self.define_varibles_calculadas()
        self.vectores_coeficientes()
        self.matrices_coeficientes_tecnicos()
        self.variables_macroeconomicas()
        
    def carga_datos(self,archivo_domestico, archivo_total):
        datosD = pd.read_excel(archivo_domestico,skiprows=5,index_col=0)
        datosT = pd.read_excel(archivo_total,skiprows=5,index_col=0)
        
        flujos_intermedios = ["111110 - Cultivo de soya","931810 - Actividades de seguridad nacional"]
        self.Zd = self.obtenerSubmatriz(flujos_intermedios,flujos_intermedios,datosD)
        self.Zt = self.obtenerSubmatriz(flujos_intermedios,flujos_intermedios,datosT)
        
        demanda_final = ["CP - Consumo Privado","YA0 - Discrepancia estadística"]
        self.Fd = self.obtenerSubmatriz(flujos_intermedios,demanda_final,datosD)
        Ft = self.obtenerSubmatriz(flujos_intermedios,demanda_final,datosT)
        self.Ft = np.delete(Ft,-2,1)
        
        self.etiSCIAN = datosD.loc[flujos_intermedios[0]:flujos_intermedios[1]].index
        eti = self.etiSCIAN.str.slice(9)
        reemp_dic = {
           "Fabricación de" : "Fab.",
           "Cultivo de" : "C.",
           "Explotación de" : "Exp.",
           "Producción" : "Prod.",
           "producción" : "prod.",
           "Minería de" : "Min.",
           "Elaboración de" : "Elab.",
           "Transporte de" : "Trans.",
           "Transporte" : "Trans.",
           "Alquiler" : "Alq.",
           "Servicios de" : "S.",
           "Servicios" : "S."
           }
        eti2 = list(self.etiSCIAN)
        for key,val in reemp_dic.items():
            eti2 = list(map(lambda s: s.replace(key, val), eti2))
        self.eti2 = eti2
        
        self.REM = self.obtenerSubmatriz(["D.1 - Remuneración de los asalariados"]*2,flujos_intermedios,datosD)
        self.PT = self.obtenerSubmatriz(["PT - Puestos de trabajo"]*2,flujos_intermedios,datosD)
        self.PTR = self.obtenerSubmatriz(["PTR - Puestos de trabajo remunerados"]*2,flujos_intermedios,datosD)
        self.PIB = self.obtenerSubmatriz(["B.1bP - Producto interno bruto"]*2,flujos_intermedios,datosD)
        
        
    def obtenerSubmatriz(self,renglones,columnas,datos):
        datos2 = datos.fillna(0)
        datos_interes = datos2.loc[renglones[0]:renglones[1],columnas[0]:columnas[1]]
        matriz = np.array(datos_interes) 
        return matriz
    
    
    def invD(self, vector):
        if vector.ndim > 1 : 
            vector = vector.flatten()
        invertida = np.where(vector==0,0,1/vector)
        diagonal = np.diag(invertida)
        return diagonal
    
    
    def define_varibles_calculadas(self):
        self.n = len(self.Zd)
        self.fd = np.sum(self.Fd,1)
        ft = np.sum(self.Ft,1)
        self.x = np.sum(self.Zd,1) + self.fd
        self.M = self.Zt - self.Zd
        Fm = self.Ft - self.Fd
        Iota = np.ones([1,self.n])
    
    
    def vectores_coeficientes(self):
        invD_x = self.invD(self.x)
        self.rem = np.matmul(self.REM,invD_x)
        self.pt = np.matmul(self.PT,invD_x)
        self.ptr = np.matmul(self.PTR,invD_x)
        self.pib = np.matmul(self.PIB,invD_x)
        imp = np.matmul(np.sum(self.M,0),invD_x)
    
    def matrices_coeficientes_tecnicos(self):
        #A = Zd . invD[x];(*Matriz de coeficientes técnicos domésticos*)
        self.A = np.matmul(self.Zd,self.invD(self.x))
        #Am = M . invD[x];(*Matriz de coeficientes técnicos de importaciones*)
        self.Am = np.matmul(self.M,self.invD(self.x))
        #At = A + Am;(*Matriz de coeficientes técnicos totales*)
        self.At = self.A + self.Am
        #L = Inverse[IdentityMatrix[n] - A];(*Matriz inversa de Leontief*)
        L = np.linalg.inv( np.identity(self.n) - self.A)
        
    def variables_macroeconomicas(self):
        #xtot = Total[x];(*Valor Bruto de la Producción total*)
        self.xtot = np.sum(self.x)
        #PIBtot = Total[PIB];(*PIB total*)
        self.PIBtot = np.sum(self.PIB)
        #REMtot = Total[REM];(*Remuneraciones totales*)
        self.REMtot = np.sum(self.REM)
        #PTtot = Total[PT];(*Puestos de trabajo totales*)
        self.PTtot = np.sum(self.PT)
        #PTRtot = Total[PTR];(*Puestos de trabajo remunerados totales*)
        self.PTRtot = np.sum(self.PTR)
        #IMPtot = Total[Total[M]];(*Importaciones totales*)
        self.IMPtot = np.sum(self.M)
        #macro0 = {xtot, PIBtot, REMtot, PTtot, PTRtot, IMPtot};(*Lista de variables macro originales*)
        self.macro0 = np.array([self.xtot, self.PIBtot, self.REMtot, self.PTtot, self.PTRtot, self.IMPtot])
    
    
    def simular_tablas(self,lista):
        etiMacro = ["Producción (millones de pesos)",    "PIB (millones de pesos)", "Remuneraciones (millones de pesos)",    "Puestos de Trabajo", "Puestos de Trabajo Remunerados",    "Importaciones (millones de pesos)"]
        
        #numSust = Length[lista];(*Número de insumos a integrar*)
        numSust = len(lista)
        
        Amod,AMmod = self.simula_adyacencia(lista)
        
        #Lmod = Inverse[ IdentityMatrix[n] -  Amod];(*Inversa de Leontief doméstica modificada*)
        Lmod = np.linalg.inv( np.identity(self.n) - Amod)
        #xmod = Lmod . fd;(*Vector de VBP modificado*)
        xmod = np.matmul(Lmod , self.fd)
        #PIBmod = pib . xmod;(*Total del PIB modificado*)
        PIBmod = np.matmul(self.pib,  xmod)[0]
        #PIBmodVec = pib*xmod;(*Vector del PIB por actividad modificado*)
        PIBmodVec = self.pib*xmod
        #REMmod = rem . xmod;(*Total de Remuneraciones modificadas*)
        REMmod = np.matmul(self.rem, xmod)[0]
        #REMmodVec = rem*xmod;(*Vector de Remuneraciones por actividad modificadas*)
        REMmodVec = self.rem*xmod
        #PTmod = pt . xmod;(*Total del PT modificados*)
        PTmod = np.matmul(self.pt , xmod)[0]
        #PTmodVec = pt*xmod;(*Vector de PT por actividad modificados*)
        PTmodVec = self.pt*xmod
        #PTRmod = ptr . xmod;(*Total del PTR modificados*)
        PTRmod = np.matmul(self.ptr , xmod)[0]
        #PTRmodVec = ptr*xmod;(*Vector del PTR por actividad modificados*)
        PTRmodVec = self.ptr*xmod
        #IMPmod = Total[AMmod] . xmod;(*Total del importaciones modificadas*)
        IMPmod = np.matmul( np.sum(AMmod,0), xmod)
        #(*-- Generación de tablas con resultados*)
        #(*Tabla de resultados macro*)
        resMacro = [np.sum(xmod) , PIBmod , REMmod , PTmod , PTRmod , IMPmod ] - self.macro0
        tablaMacro = pd.DataFrame({"Variable":etiMacro,
                                   "Niveles originales (millones de pesos)":self.macro0,
                                   "Variación (millones de pesos)":resMacro.flatten(),
                                   "Variación porcentual (%)":(resMacro/self.macro0*100).flatten()})
        #(*Tabla de principales sectores afectados vía PIB*)
        tablaPIB = pd.DataFrame({"Variable":self.etiSCIAN, 
                                 "Niveles originales (millones de pesos)":self.PIB.flatten(), 
                                 "Variación (millones de pesos)":(PIBmodVec - self.PIB).flatten(),
                                 "Variación porcentual (%)":((np.matmul(PIBmodVec , self.invD(self.PIB)) - 1)*100).flatten()})

        #(*Tabla de principales sectores afectados vía Remuneraciones*)
        tablaREM = pd.DataFrame({"Variable":self.etiSCIAN, 
                                 "Niveles originales (millones de pesos)":self.REM.flatten(), 
                                 "Variación (millones de pesos)":(REMmodVec - self.REM).flatten(), 
                                 "Variación porcentual (%)":((np.matmul(REMmodVec , self.invD(self.REM)) - 1)*100).flatten()})
        
        #(*Tabla de principales sectores afectados vía Puestos de Trabajo Remunerados*)
        tablaPTR = pd.DataFrame({"Variable":self.etiSCIAN, 
                                 "Niveles originales (Puestos)":self.PTR.flatten(), 
                                 "Variación (Puestos)":(PTRmodVec - self.PTR).flatten(), 
                                 "Variación porcentual (%)":((np.matmul(PTRmodVec , self.invD(self.PTR)) - 1)*100).flatten()})
        
        #(*Tabla de principales sectores afectados vía Puestos de Trabajo*)
        tablaPT = pd.DataFrame({"Variable":self.etiSCIAN, 
                                "Niveles originales (Puestos)":self.PT.flatten(), 
                                "Variación (Puestos)":(PTmodVec - self.PT).flatten(),  
                                "Variación porcentual (%)":((np.matmul(PTmodVec , self.invD(self.PT)) - 1)*100).flatten()})
        
        return [tablaMacro, tablaPIB, tablaREM, tablaPTR, tablaPT]
    
    def simula_adyacencia(self,lista):
        #Amod = ReplacePart[A, listaSustA];(*Matriz de coeficientes técnicos modificada*)
        #AMmod = ReplacePart[Am, listaSustAm];(*Matriz de coeficientes importados modificada*)
        Amod = self.A.copy()
        AMmod = self.Am.copy()
        for elem in lista:
            Amod[elem[0],elem[1]] += (elem[2]/100)*self.Am[elem[0],elem[1]]
            AMmod[elem[0],elem[1]] = (1 - elem[2]/100)* self.Am[elem[0],elem[1]]
            
        return [Amod,AMmod]
    
    def crea_subred (self, sector_inicial, matriz_adyacencia, filtro=0.04):        
        DG = nx.DiGraph()
        DG.add_edges_from([(i,j) 
                            for i, origen in enumerate(matriz_adyacencia) 
                            for j, destino in enumerate(origen) 
                            if destino > filtro and i != j and not i in [446, 447]])
        
        
        try:
            SDG = DG.subgraph(nx.bfs_tree(DG,sector_inicial,True).nodes)
        except nx.NetworkXError:
            SDG = nx.DiGraph()
            SDG.add_node(sector_inicial)
            
        for n in SDG.nodes:
            SDG.nodes[n]["etiqueta"] = self.etiSCIAN[n]
        
        return SDG
    
    def simula_red(self, sector_inicial, lista, filtro=0.04):
        red_domestica = self.crea_subred (sector_inicial, self.A, filtro)
        red_total = self.crea_subred (sector_inicial, self.At, filtro)
        Amod,AMmod = self.simula_adyacencia(lista)
        red_simulada = self.crea_subred (sector_inicial, Amod, filtro)
        
        for e in red_total.edges:
            if red_domestica.has_edge(*e):
                tipo = "domestico"
            elif any([l[0] == e[0] and l[1] == e[1] for l in lista]):
                tipo = "simulado_directo"
            elif red_simulada.has_edge(*e):
                tipo = "simulado_indirecto"
            else:
                tipo = "total"
            red_total.edges[e]["tipo"] = tipo
            
        for n in red_total.nodes:
            if n == sector_inicial:
                tipo = "inicial"
            elif red_domestica.has_node(n):
                tipo = "domestico"
            else:
                tipo = "total"
            red_total.nodes[n]["tipo"] = tipo
                
            
        return red_total



In [205]:
simulador = Simulador_MIP("../data/mip_d_pb_pxp__4_22_42_202_133.xlsx","../data/mip_t_pb_pxp__4_22_45_202_133.xlsx" )

In [196]:
A, B, C, D, E = simulador.simular_tablas([[284,8,100]])

<ipython-input-176-75d394726961>:59: RuntimeWarning: divide by zero encountered in true_divide
  invertida = np.where(vector==0,0,1/vector)


In [ ]:
A

In [ ]:
B.sort_values("Variación (millones de pesos)",0,False)

In [206]:
dg = simulador.simula_red(51, [[284,8,100]])

In [207]:
nx.node_link_data(dg)

{'directed': True,
 'multigraph': False,
 'graph': {},
 'nodes': [{'etiqueta': '111110 - Cultivo de soya', 'tipo': 'total', 'id': 0},
  {'etiqueta': '111129 - Cultivo anual de otras semillas oleaginosas',
   'tipo': 'total',
   'id': 3},
  {'etiqueta': '111151 - Cultivo de maíz grano', 'tipo': 'domestico', 'id': 8},
  {'etiqueta': '111191 - Cultivo de sorgo grano',
   'tipo': 'domestico',
   'id': 11},
  {'etiqueta': '324110 - Refinación de petróleo',
   'tipo': 'domestico',
   'id': 272},
  {'etiqueta': '325110 - Fabricación de petroquímicos básicos del gas natural y del petróleo refinado',
   'tipo': 'domestico',
   'id': 276},
  {'etiqueta': '325120 - Fabricación de gases industriales',
   'tipo': 'total',
   'id': 277},
  {'etiqueta': '325180 - Fabricación de otros productos químicos básicos inorgánicos',
   'tipo': 'total',
   'id': 279},
  {'etiqueta': '325190 - Fabricación de otros productos químicos básicos orgánicos',
   'tipo': 'total',
   'id': 280},
  {'etiqueta': '325310 -